# Syntactic analysis

## 1. Explain the meaning of the three UD tags of syntactic relations using a fragment of a marked-up corpus as an example

In [ ]:
! wget -q https://www.dropbox.com/s/am6nasx6bx82nhp/RuEval2017-Lenta-news-dev.conllu

In [ ]:
! head -497 RuEval2017-Lenta-news-dev.conllu | tail -10

21	в	в	ADP	_	_	23	case	_	_
22	1969	1969	ADJ	_	NumForm=Digit	23	amod	_	_
23	году	год	NOUN	_	Animacy=Inan|Case=Loc|Gender=Masc|Number=Sing	20	obl	_	_
24	за	за	ADP	_	_	25	case	_	_
25	задержание	задержание	NOUN	_	Animacy=Inan|Case=Acc|Gender=Neut|Number=Sing	20	obl	_	_
26	особо	особо	ADV	_	Degree=Pos	27	advmod	_	_
27	опасного	опасный	ADJ	_	Case=Gen|Degree=Pos|Gender=Masc|Number=Sing	28	amod	_	_
28	преступника	преступник	NOUN	_	Animacy=Anim|Case=Gen|Gender=Masc|Number=Sing	25	nmod	_	_
29	.	.	PUNCT	_	_	9	punct	_	_



In [ ]:
corpus_file = 'RuEval2017-Lenta-news-dev.conllu'

In [ ]:
! pip install -q natasha
! pip install -q conllu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 68.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
from natasha.syntax import NewsSyntaxParser
from natasha import (
    Doc, 
    Segmenter,
    NewsEmbedding,
    # MorphVocab,
    # NewsMorphTagger
)
from conllu import parse
import string

# Create instances of needed Natasha components
segmenter = Segmenter()
emb = NewsEmbedding()
syntax_parser = NewsSyntaxParser(emb)
# morph_vocab = MorphVocab()
# morph_tagger = NewsMorphTagger(emb)

# Download CoNLL-U
with open(corpus_file, 'r', encoding='utf-8') as file:
    conll_text = file.read()

# Parse CoNLL-U text and create Doc object from tokens
parsed_data = parse(conll_text)
# tokens = [token['form'] for sentence in parsed_data for token in sentence if token['form'] not in string.punctuation]
tokens = [token['form'] for sentence in parsed_data for token in sentence]
print(tokens[5:8])
doc = Doc(' '.join(tokens))
# Tokenization
doc.segment(segmenter)
doc.parse_syntax(syntax_parser)
doc.sents[0].syntax.print()
# print(doc.tokens[5:8])
# # Setting morphological tags for tokens
# doc.tag_morph(morph_tagger)
# # Getting tokens and morphological tags
# for token in doc.tokens:
#     print("Токен: ", token.text)
#     print("Тег: ", token.pos)
#     print("Тег морфологии: ", token.feats)
#     print("--------------")

['подробности', 'программы', ',']
    ┌──► Официальные    amod
    │ ┌► американские   amod
    └─└─ власти         nsubj
┌───└─┌─ отказываются   
│ ┌─┌─└► комментировать xcomp
│ │ └►┌─ подробности    obj
│ │   └► программы      nmod
│ │   ┌► ,              punct
│ └──►└─ ссылаясь       advcl
│ │ ┌──► на             case
│ │ │ ┌► ее             det
│ └►└─└─ секретность    obl
└──────► .              punct


## 2. Write a function for splitting a two-part compound sentence into simple ones

### razdel

In [ ]:
! pip install razdel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
text = '''
- "Так в чем же дело?" - "Не ра-ду-ют".
И т. д. и т. п. В общем, вся газета
'''

from razdel import sentenize
print(list(sentenize(text)))
# print(list(sentenize(doc.sents[0].text)))

[Substring(1, 23, '- "Так в чем же дело?"'), Substring(24, 40, '- "Не ра-ду-ют".'), Substring(41, 56, 'И т. д. и т. п.'), Substring(57, 76, 'В общем, вся газета')]


In [ ]:
sent1 = doc.sents[0]
sent2 = doc.sents[1]
print(sent1.text)
print(sent2.text)

Официальные американские власти отказываются комментировать подробности программы , ссылаясь на ее секретность .
Единственный сын одного из высокопоставленных северокорейских генералов бежал из страны вместе с семьей и сейчас находится в руках американской разведки .


### spacy

In [ ]:
! pip install -q spacy
! python -m spacy download ru_core_news_sm --quiet

2023-04-19 00:02:29.129034: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [ ]:
import spacy

nlp = spacy.load("ru_core_news_sm")

doc_spacy = nlp(sent1.text)

simple_sentences = [sent.text for sent in doc_spacy.sents]

for i, simple_sentence in enumerate(simple_sentences):
    print(f"Простое предложение {i + 1}: {simple_sentence}")

Простое предложение 1: Официальные американские власти отказываются комментировать подробности программы , ссылаясь на ее секретность .


### custom

In [ ]:
import string

def split_complex_sentences(doc):
    # Split complex sentences into simple clauses
    j = 0
    split_sentences = []
    for sent in doc.sents:
        clauses = []
        conj_indices = [i for i, token in enumerate(sent.tokens) if token.rel == 'conj']
        if conj_indices:
            print(sent.text)
            clause_start = 0
            for conj_index in conj_indices:
                clause_end = conj_index
                clause = ' '.join(token.text for token in sent.tokens[clause_start:clause_end])
                if clause:
                    clauses.append(clause.strip())
                clause_start = conj_index
            last_clause = ' '.join(token.text for token in sent.tokens[conj_indices[-1]:])
            if last_clause:
                clauses.append(last_clause.strip())
            split_sentences.extend(clauses)
            for i, clause in enumerate(clauses):
              print(r'Simple sentence {}: {}'.format(i + 1, clause))
            j += 1
            if j > 9:
              break
            print('')

    return split_sentences

In [ ]:
split_sentences = split_complex_sentences(doc)

Единственный сын одного из высокопоставленных северокорейских генералов бежал из страны вместе с семьей и сейчас находится в руках американской разведки .
Simple sentence 1: Единственный сын одного из высокопоставленных северокорейских генералов бежал из страны вместе с семьей и сейчас
Simple sentence 2: находится в руках американской разведки .

Это решение уже трижды рассматривалось в судах — первая инстанция удовлетворила жалобу " ЮКОСа " , однако вторая и третья признали арест акций законным .
Simple sentence 1: Это решение уже трижды рассматривалось в судах — первая инстанция удовлетворила жалобу " ЮКОСа " , однако вторая и
Simple sentence 2: третья признали арест акций законным .

Ранее один из преступников , представившийся " Хасаном " , заявил , что он " русский и требует самолет в Россию " .
Simple sentence 1: Ранее один из преступников , представившийся " Хасаном " , заявил , что он " русский и
Simple sentence 2: требует самолет в Россию " .

Министр общественного порядка так

## 3. Write a function for finding the smallest common ancestor of two tokens in a dependency tree

In [ ]:
from natasha import Doc, MorphVocab
from natasha.syntax import NewsSyntaxParser

segmenter = Segmenter()
emb = NewsEmbedding()
syntax_parser = NewsSyntaxParser(emb)

# Функция для нахождения наименьшего общего предка (LCA) в дереве зависимостей
def find_lca(doc, token1, token2):
    # Найдите узел, соответствующий каждому из токенов
    node1 = None
    node2 = None
    for sent in doc.sents:
        for token in sent.tokens:
            # print(token.text, token1)
            if token.text == token1.text:
                node1 = token
            if token.text == token2.text:
                node2 = token

    # Проверка на наличие узлов в дереве зависимостей
    if node1 is None or node2 is None:
        raise ValueError("Один из токенов не найден в дереве зависимостей")

    # Найдите путь от корня до каждого из узлов
    path1 = set()
    path2 = set()
    while node1 is not None:
        path1.add(node1)
        node1 = find_token_head(doc, node1)
    while node2 is not None:
        path2.add(node2)
        node2 = find_token_head(doc, node2)

    # Найдите наименьший общий элемент в списках путей
    lca = None
    for node in path1:
        if node in path2:
            lca = node
            break

    return lca

def find_token_head(doc, node):
    for token in doc.tokens:
        if token.id == node.head_id:
            return token
    return None

# Пример использования функции
text = "Единственный сын одного из высокопоставленных северокорейских генералов бежал из страны вместе с семьей и сейчас находится в руках американской разведки."
doc = Doc(text)
doc.segment(segmenter)
doc.parse_syntax(syntax_parser)
doc.sents[0].syntax.print()

token1_id = 14
token2_id = 17

token1 = doc.tokens[token1_id]
token2 = doc.tokens[token2_id]

lca = find_lca(doc, token1, token2)

if lca is not None:
    print(f"Наименьший общий предок для токенов {token1.text} и {token2.text}: {lca.text}")
else:
    print("Наименьший общий предок не найден")


            ┌► Единственный       amod
    ┌────►┌─└─ сын                nsubj
    │ ┌───└──► одного             nmod
    │ │ ┌────► из                 case
    │ │ │ ┌──► высокопоставленных amod
    │ │ │ │ ┌► северокорейских    amod
    │ └►└─└─└─ генералов          nmod
┌─┌─└───┌─┌─── бежал              
│ │     │ │ ┌► из                 case
│ │     │ └►└─ страны             obl
│ │     └►┌─── вместе             advmod
│ │       │ ┌► с                  case
│ │       └►└─ семьей             obl
│ │       ┌──► и                  cc
│ │       │ ┌► сейчас             advmod
│ └────►┌─└─└─ находится          conj
│       │   ┌► в                  case
│     ┌─└──►└─ руках              obl
│     │     ┌► американской       amod
│     └────►└─ разведки           nmod
└────────────► .                  punct
Наименьший общий предок для токенов сейчас и руках: находится


## 4. Compare three pairs of sentences using two methods: comparing dependency tree editing distance (zss) and a cosine measure between BERT-embeddings

In [ ]:
! pip install -q zss

  Preparing metadata (setup.py) ... done


In [152]:
import spacy
from zss import simple_distance, Node

nlp = spacy.load("ru_core_news_sm")

In [ ]:
examples = ["Привет, у нас на кухне нашли плесень!", 
            "На нашей кухне нашли много всего: бактерии, грибки и позавчерашнее молоко.",
            "Привет, у них в подвале нашли клад!" ]

In [210]:
# from natasha import MorphAnalyzer, SyntaxParser, NewsSyntaxParser
from natasha import (
    Doc, 
    Segmenter,
    NewsEmbedding,
    MorphVocab,
    NewsMorphTagger
)
from zss import Node, simple_distance

segmenter = Segmenter()
emb = NewsEmbedding()
syntax_parser = NewsSyntaxParser(emb)
morph_vocab = MorphVocab()

def analyze_postags(sent):
    """ Выполняем морфологический анализ предложения и возвращаем список постегов """
    words = sent.split()
    postags = []
    for word in words:
        parsed = morph_vocab.parse(word)[0]
        postag = parsed.tag.POS
        postags.append(postag)
    return postags

def analyze_syntax(sent):
    """ Выполняем синтаксический анализ предложения и возвращаем список синтаксических аннотаций слов """
    # syntax_dep_tree = syntax_parser.parse(sent).as_json
    doc = Doc(sent)
    doc.segment(segmenter)
    doc.parse_syntax(syntax_parser)
    # syntax_dep_tree = doc.sents[0].syntax.tokens[0].as_json
    syntax_dep_tree = doc.sents[0].syntax.tokens
    # print(syntax_dep_tree)

    return syntax_dep_tree

def pos_dep_tree(postags, syntax_dep_tree):
    """ Конвертируем результат морфосинтаксического анализа в zss-дерево из частей речи и синтаксических связей
    NOUN -> advmod -> VERB """
    
    root = Node('root')
    pos_nodes = {}
    for i, postag in enumerate(postags):
        pos_nodes[i] = Node(postag)
    
    for i, dependency_edge in enumerate(syntax_dep_tree):
        relation = dependency_edge.rel
        index = dependency_edge.id
        parent_index = dependency_edge.head_id
        
        relation_node = Node(relation)
        if parent_index in pos_nodes:
            pos_nodes[parent_index].addkid(relation_node)
        else:
            root.addkid(relation_node)
        
        if index in pos_nodes:
            relation_node.addkid(pos_nodes[index])
        else:
            relation_node.addkid(Node(syntax_dep_tree[i].text))
    
    return root

def sent_dep_tree(sent):
    """ Получаем список постегов и список синтаксических аннотаций слов предложения, полученные выбранным анализатором (или анализаторами) """
    postags = analyze_postags(sent)
    syntax_dep_tree = analyze_syntax(sent)
    return pos_dep_tree(postags, syntax_dep_tree)

def dep_tree_similarity(dep1, dep2, smoothing=5.0):
    return smoothing / (smoothing + simple_distance(dep1, dep2))

def sentence_similarity(sent1, sent2, smoothing=5.0):
    return dep_tree_similarity(sent_dep_tree(sent1), sent_dep_tree(sent2), smoothing)

examples = ["Привет, у нас на кухне нашли плесень!", 
            "На нашей кухне нашли много всего: бактерии, грибки и позавчерашнее молоко.",
            "Привет, у них в подвале нашли клад!" ]

for pair in [[0, 1], [1, 2], [0, 2]]:
    print((examples[pair[0]], examples[pair[1]],
           sentence_similarity(examples[pair[0]], examples[pair[1]])))
    
print('')
print('Control:')
examples = ["Привет, у нас на кухне нашли плесень!", 
            "На нашей кухне нашли много всего: бактерии, грибки и позавчерашнее молоко.",
            "Привет, у нас на кухне нашли плесень!" ]
            
for pair in [[0, 1], [1, 2], [0, 2]]:
    print((examples[pair[0]], examples[pair[1]],
           sentence_similarity(examples[pair[0]], examples[pair[1]])))


('Привет, у нас на кухне нашли плесень!', 'На нашей кухне нашли много всего: бактерии, грибки и позавчерашнее молоко.', 0.041666666666666664)
('На нашей кухне нашли много всего: бактерии, грибки и позавчерашнее молоко.', 'Привет, у них в подвале нашли клад!', 0.04065040650406504)
('Привет, у нас на кухне нашли плесень!', 'Привет, у них в подвале нашли клад!', 0.14705882352941177)

Control:
('Привет, у нас на кухне нашли плесень!', 'На нашей кухне нашли много всего: бактерии, грибки и позавчерашнее молоко.', 0.041666666666666664)
('На нашей кухне нашли много всего: бактерии, грибки и позавчерашнее молоко.', 'Привет, у нас на кухне нашли плесень!', 0.041666666666666664)
('Привет, у нас на кухне нашли плесень!', 'Привет, у нас на кухне нашли плесень!', 1.0)


### embeddings

In [183]:
! pip install -q transformers

In [184]:
from transformers import BertTokenizer, BertModel
import torch

# Для примера возьмём не оригинальный BERT (DeepPavlov/rubert-base-cased), 
# а sentence-BERT, дообученный на задаче языкового вывода, и, следовательно, 
# более точный в семантическом представлении предложений.
# https://huggingface.co/DeepPavlov/rubert-base-cased-sentence#rubert-base-cased-sentence
# Можете заменить на любой другой вариант открытой модели BERT для русского языка.

tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased-sentence')
model = BertModel.from_pretrained('DeepPavlov/rubert-base-cased-sentence')

In [212]:
import numpy
from sklearn.metrics.pairwise import cosine_similarity

def embed_sentence(sentence: str):
  """ Возвращает векторное представление предложения с использованием модели BERT """
  # Токенизация предложения
  input_ids = torch.tensor([tokenizer.encode(sentence, add_special_tokens=True)])
  # Предсказания модели
  outputs = model(input_ids)
  # Возвращаем усредненное векторное представление
  return torch.mean(outputs[0].detach(), axis=1).numpy()

embedded_examples = [embed_sentence(sent) for sent in examples]

for pair in [[0, 1], [1, 2], [0, 2]]:
  print((examples[pair[0]], examples[pair[1]],
    cosine_similarity(embedded_examples[pair[0]], embedded_examples[pair[1]])[0][0]))
  
print('')
print('Control:')
examples = ["Привет, у нас на кухне нашли плесень!", 
            "На нашей кухне нашли много всего: бактерии, грибки и позавчерашнее молоко.",
            "Привет, у нас на кухне нашли плесень!" ]

for pair in [[0, 1], [1, 2], [0, 2]]:
  print((examples[pair[0]], examples[pair[1]],
    cosine_similarity(embedded_examples[pair[0]], embedded_examples[pair[1]])[0][0]))

('Привет, у нас на кухне нашли плесень!', 'На нашей кухне нашли много всего: бактерии, грибки и позавчерашнее молоко.', 0.78814584)
('На нашей кухне нашли много всего: бактерии, грибки и позавчерашнее молоко.', 'Привет, у нас на кухне нашли плесень!', 0.78814584)
('Привет, у нас на кухне нашли плесень!', 'Привет, у нас на кухне нашли плесень!', 1.0000001)

Control:
('Привет, у нас на кухне нашли плесень!', 'На нашей кухне нашли много всего: бактерии, грибки и позавчерашнее молоко.', 0.78814584)
('На нашей кухне нашли много всего: бактерии, грибки и позавчерашнее молоко.', 'Привет, у нас на кухне нашли плесень!', 0.78814584)
('Привет, у нас на кухне нашли плесень!', 'Привет, у нас на кухне нашли плесень!', 1.0000001)
